# Gallery Example: M/M/1 Tandem Network

This example demonstrates a simple tandem network of two M/M/1 queues in series:
- **Topology**: 2 queues in series (tandem)
- **Arrivals**: Poisson process to first queue
- **Service**: Exponential service times at both stations
- **Servers**: 1 server at each station
- **Scheduling**: FCFS at both stations

This is a fundamental network topology used to model sequential processing stages.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
fun gallery_mm1_tandem(): Network {    """Create simple M/M/1 tandem network (2 queues in series)"""    # Use the linear model with n=2 and default utilization pattern    from gallery_mm1_linear import gallery_mm1_linear  # Import would work in practice        # Alternative: create directly for better control    model = Network("M/M/1-Tandem")        # Block 1: nodes    source = Source(model, "mySource")    queue1 = Queue(model, "Queue1", SchedStrategy.FCFS)    queue2 = Queue(model, "Queue2", SchedStrategy.FCFS)    sink = Sink(model, "mySink")        # Block 2: classes    oclass = OpenClass(model, "myClass")    source.setArrival(oclass, Exp(1))  # λ = 1        # Symmetric utilization: both stations have similar load    queue1.setService(oclass, Exp.fitMean(0.4))  # Service time = 0.4, so ρ₁ = 0.4    queue2.setService(oclass, Exp.fitMean(0.3))  # Service time = 0.3, so ρ₂ = 0.3        # Block 3: topology - serial routing    P = model.initRoutingMatrix()    P.addRoute(oclass, source, queue1, 1.0)    P.addRoute(oclass, queue1, queue2, 1.0)    P.addRoute(oclass, queue2, sink, 1.0)    model.link(P)        return model# Create the modelmodel = gallery_mm1_tandem()println(f"Topology: Source -> Queue1 -> Queue2 -> Sink")

## Theoretical Analysis for Tandem Network

For a tandem network with arrival rate λ and service times s₁, s₂:
- **Throughput**: Same at all stations (λ = 1)
- **Utilization**: ρᵢ = λ × sᵢ
  - ρ₁ = 1 × 0.4 = 0.4
  - ρ₂ = 1 × 0.3 = 0.3
- **Response Time**: W_total = W₁ + W₂
- **Queue Length**: L_total = L₁ + L₂

Since queues are independent (product-form network):
- W₁ = s₁/(1-ρ₁) = 0.4/(1-0.4) = 0.667
- W₂ = s₂/(1-ρ₂) = 0.3/(1-0.3) = 0.429
- W_total = 0.667 + 0.429 = 1.096

In [ ]:
// Solve with multiple solvers
println("\n=== Solver Results ===")
// MVA Solver
val solver_mva = MVA(model)
val avg_table_mva = solver_mva.avgTable
println("\nMVA Solver:")
println(avg_table_mva)
// CTMC Solver
val solver_ctmc = CTMC(model, "cutoff", 10)
val avg_table_ctmc = solver_ctmc.avgTable
println("\nCTMC Solver:")
println(avg_table_ctmc)
// Fluid Solver
val solver_fluid = FLD(model)
val avg_table_fluid = solver_fluid.avgTable
println("\nFluid Solver:")
println(avg_table_fluid)

In [ ]:
// Verify theoretical predictions
println("\n=== Theoretical vs Simulation ===")

val solver = MVA(model)
val avg_table = solver.avgTable
// Extract results for Queue1 and Queue2 (skip source and sink)
val queue1_util = float(avg_table.iloc[1, 1])  # Queue1 utilization
val queue1_resp = float(avg_table.iloc[1, 2])  # Queue1 response time
val queue1_length = float(avg_table.iloc[1, 3])  # Queue1 queue length

val queue2_util = float(avg_table.iloc[2, 1])  # Queue2 utilization
val queue2_resp = float(avg_table.iloc[2, 2])  # Queue2 response time
val queue2_length = float(avg_table.iloc[2, 3])  # Queue2 queue length
// Theoretical values
val rho1_theory = 0.4
val rho2_theory = 0.3
val w1_theory = 0.4 / (1 - 0.4)
val w2_theory = 0.3 / (1 - 0.3)
val w_total_theory = w1_theory + w2_theory

println(f"Queue 1:")
println(f"  Utilization: Theory={rho1_theory:.3f}, Simulation={queue1_util:.3f}")
println(f"  Response Time: Theory={w1_theory:.3f}, Simulation={queue1_resp:.3f}")

println(f"\nQueue 2:")
println(f"  Utilization: Theory={rho2_theory:.3f}, Simulation={queue2_util:.3f}")
println(f"  Response Time: Theory={w2_theory:.3f}, Simulation={queue2_resp:.3f}")

println(f"\nTotal Response Time: Theory={w_total_theory:.3f}, Simulation={queue1_resp + queue2_resp:.3f}")

In [ ]:
// Analyze bottleneck effects
println("\n=== Bottleneck Analysis ===")

fun create_tandem_with_bottleneck(bottleneck_position, service_times): Network {
    """Create tandem network with specified bottleneck"""
    val model_bn = Network(f"Tandem-Bottleneck{bottleneck_position}")
    val source = Source(model_bn, "Source")
    val queue1 = Queue(model_bn, "Queue1", SchedStrategy.FCFS)
    val queue2 = Queue(model_bn, "Queue2", SchedStrategy.FCFS)
    val sink = Sink(model_bn, "Sink")
    
    val oclass = OpenClass(model_bn, "Class")
    source.setArrival(oclass, Exp(1))
    queue1.setService(oclass, Exp.fitMean(service_times[0]))
    queue2.setService(oclass, Exp.fitMean(service_times[1]))
    
    val P = model_bn.initRoutingMatrix()
    P.addRoute(oclass, source, queue1, 1.0)
    P.addRoute(oclass, queue1, queue2, 1.0)
    P.addRoute(oclass, queue2, sink, 1.0)
    model_bn.link(P)
    
    return model_bn
// Test different bottleneck scenarios
val scenarios = [
    ("Balanced", [0.3, 0.3]),
    ("Queue1 Bottleneck", [0.8, 0.2]),
    ("Queue2 Bottleneck", [0.2, 0.8])
]

for name, service_times in scenarios:
    val model_scenario = create_tandem_with_bottleneck(1 if service_times[0] > service_times[1] else 2, service_times)
    val solver = MVA(model_scenario)
    val avg_table = solver.avgTable
    
    val q1_util = float(avg_table.iloc[1, 1])
    val q2_util = float(avg_table.iloc[2, 1])
    val total_resp = float(avg_table.iloc[1, 2]) + float(avg_table.iloc[2, 2])
    
    println(f"{name}: ρ₁={q1_util:.3f}, ρ₂={q2_util:.3f}, W_total={total_resp:.3f}")